## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from env import API_KEY

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Saskylakh,RU,2022-09-15 15:17:18,71.9167,114.0833,30.90,94,85,5.70,overcast clouds
1,1,Rikitea,PF,2022-09-15 15:17:19,-23.1203,-134.9692,70.02,69,0,9.53,clear sky
2,2,Mataura,NZ,2022-09-15 15:17:19,-46.1927,168.8643,46.47,85,100,7.16,overcast clouds
3,3,Nueva Guinea,NI,2022-09-15 15:17:20,11.6876,-84.4562,78.10,88,59,6.02,broken clouds
4,4,Vaini,TO,2022-09-15 15:17:20,-21.2000,-175.2000,73.56,94,40,4.61,scattered clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 95


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
cities_preferred_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]
cities_preferred_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Rikitea,PF,2022-09-15 15:17:19,-23.1203,-134.9692,70.02,69,0,9.53,clear sky
3,3,Nueva Guinea,NI,2022-09-15 15:17:20,11.6876,-84.4562,78.10,88,59,6.02,broken clouds
4,4,Vaini,TO,2022-09-15 15:17:20,-21.2000,-175.2000,73.56,94,40,4.61,scattered clouds
5,5,Jalu,LY,2022-09-15 15:17:20,29.0331,21.5482,91.49,21,0,5.79,clear sky
11,11,Thinadhoo,MV,2022-09-15 15:17:23,0.5333,72.9333,81.79,68,17,4.00,few clouds


In [7]:
# 4a. Determine if there are any empty rows.
print(len(cities_preferred_df))
cities_preferred_df.count()

343


City_ID                343
City                   343
Country                342
Date                   343
Lat                    343
Lng                    343
Max Temp               343
Humidity               343
Cloudiness             343
Wind Speed             343
Current Description    343
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = cities_preferred_df.dropna(axis="index", how="any")
print(len(clean_df))
clean_df.head()

342


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Rikitea,PF,2022-09-15 15:17:19,-23.1203,-134.9692,70.02,69,0,9.53,clear sky
3,3,Nueva Guinea,NI,2022-09-15 15:17:20,11.6876,-84.4562,78.10,88,59,6.02,broken clouds
4,4,Vaini,TO,2022-09-15 15:17:20,-21.2000,-175.2000,73.56,94,40,4.61,scattered clouds
5,5,Jalu,LY,2022-09-15 15:17:20,29.0331,21.5482,91.49,21,0,5.79,clear sky
11,11,Thinadhoo,MV,2022-09-15 15:17:23,0.5333,72.9333,81.79,68,17,4.00,few clouds


In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Rikitea,PF,70.02,clear sky,-23.1203,-134.9692,
3,Nueva Guinea,NI,78.10,broken clouds,11.6876,-84.4562,
4,Vaini,TO,73.56,scattered clouds,-21.2000,-175.2000,
5,Jalu,LY,91.49,clear sky,29.0331,21.5482,
11,Thinadhoo,MV,81.79,few clouds,0.5333,72.9333,
20,Porto Velho,BR,82.45,broken clouds,-8.7619,-63.9039,
21,Pierre,US,71.06,clear sky,44.3683,-100.3510,
24,Mahebourg,MU,71.89,scattered clouds,-20.4081,57.7000,
25,Bathsheba,BB,88.41,scattered clouds,13.2167,-59.5167,
26,Bambous Virieux,MU,71.85,light rain,-20.3428,57.7575,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": API_KEY
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping the city.")
        hotel_df.loc[index, "Hotel Name"] = np.nan 
        

Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.


In [15]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Rikitea,PF,70.02,clear sky,-23.1203,-134.9692,People ThankYou
3,Nueva Guinea,NI,78.10,broken clouds,11.6876,-84.4562,Posadita La Reyna
4,Vaini,TO,73.56,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
5,Jalu,LY,91.49,clear sky,29.0331,21.5482,صاحبة الفخامة
20,Porto Velho,BR,82.45,broken clouds,-8.7619,-63.9039,Ecos Hotel Conforto
21,Pierre,US,71.06,clear sky,44.3683,-100.3510,Ramkota Hotel
24,Mahebourg,MU,71.89,scattered clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
25,Bathsheba,BB,88.41,scattered clouds,13.2167,-59.5167,Atlantis Hotel
26,Bambous Virieux,MU,71.85,light rain,-20.3428,57.7575,Casa Tia Villa
27,Umm Lajj,SA,85.51,clear sky,25.0213,37.2685,Moon light Furnished Units


In [16]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [71]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [72]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))